<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />
  
# Capacidades de Analítica Avanzada en la Base de Datos 
# (In-Database)

## Caso 3: Clasificación de Textos
    
![Slide](images/Diapositiva4.PNG)

![Slide](images/Diapositiva8.PNG)

![Slide](images/Diapositiva5.PNG)

![Slide](images/Diapositiva6.PNG)


## **Instalar las librerías**

In [ ]:
#!pip install teradataml==17.20.0.4 kds==0.1.3 lightgbm==4.0.0 nyoka==4.3.0

## **Carga de Modulos**

In [1]:
import pandas as pd
import numpy as np
import getpass as gp
import plotly.express as px
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import kds

from teradataml import *
from teradataml.analytics.valib import *
configure.val_install_location = "val"

In [ ]:
con=create_context(host = "20.172.147.24", username="pocuser", password = gp.getpass())

![Slide](images/Diapositiva10.PNG)

## **Lectura Inicial de base de datos**

In [3]:
# Leemos la data de desarrollo de modelos
tdf = DataFrame("caso3_data_texto")

In [4]:
# Primeros Registros
tdf.head(10)

detalle,target
"301036BARRA CONST.5/8 X 9 MTS""",G
![CDATA[CARTUCHO FILTRO 2091 POLVO P100 3M]],G
"![CDATA[GUANTE QUIM. LATEX NEGRO Y NARANJA 12\ C35 T.9 MASTER]]""",G
"![CDATA[PROTECTOR T COPA ADAPT CASCO H9P3E PELTOR OPTIME 98 3M]] ![CDATA[GUANTE BADANA BLANCO S REFUERZO GC14 T.9]] ![CDATA[GUANTE QUIM. NITRILO 18\ 37-185 T.9 VERDE ANSELL EDMON SOLVEX]] ![CDATA[LENTE ANTIFOG TURBINE L. OSCURA STEELPRO]] ![CDATA[LENTE SIMPLE VISION L. CLARA CLUTE]] ![CDATA[TRAJE CONTRA PARTIC. Y LIQ. S110 T.XL STEELPRO]] ![CDATA[LENTE SOBRELENTE OTG L. CLARA CLUTE]] ![CDATA[MPU ACERO WERK T.38]]""",G
![CDATA[RESPIRADOR DESCART. POLVO KN95 MAYFIELD X10UD]],G
#TRIPTOFANO (BOLSA),G
![CDATA[RESPIRADOR DESCART. POLVO KN95 MAYFIELD X10UD]],G
"ETIQ.AUTOAD.160MM X 70MM POLIPROPILENO IMPRESO \"" 10KG NAZCA VALLEY\""""",C
"301036BARRA CONST.3/8 X 9 MTS""",G
"301036BARRA CONST.3/4 X 9 MTS""",G


In [5]:
# Vemos la dimensionalidad
tdf.shape

(192734, 2)

![Slide](images/Diapositiva6.PNG)

## **Exploración de los datos**

In [6]:
# Estadísticas Descriptivas por Columna
objcs = ColumnSummary(data=tdf,target_columns=['detalle', 'target'])
objcs.result.head()

ColumnName,Datatype,NonNullCount,NullCount,BlankCount,ZeroCount,PositiveCount,NegativeCount,NullPercentage,NonNullPercentage
target,VARCHAR(15000) CHARACTER SET LATIN,192734,0,0,None,None,None,0.0,100.0
detalle,VARCHAR(15000) CHARACTER SET LATIN,192734,0,0,None,None,None,0.0,100.0


In [7]:
objv = valib.Values(data=tdf, columns=['detalle', 'target'])
objv.result

xdb,xtbl,xcol,xtype,xcnt,xnull,xunique,xblank,xzero,xpos,xneg
POCUSER,caso3_data_texto,target,VARCHAR(15000) CHARACTER SET LATIN,192734.0,0.0,19.0,0.0,None,None,None
POCUSER,caso3_data_texto,detalle,VARCHAR(15000) CHARACTER SET LATIN,192734.0,0.0,96506.0,0.0,None,None,None


In [8]:
# Agregando show_query() al final, podemos ver que efectivamente, los códigos se traducen automáticamente a lenguaje SQL antes de ejecutarse
objcs.result.show_query()

'select * from "POCUSER"."ml__td_sqlmr_out__1697330259626198"'

In [9]:
## Calculando la proporción del Target
tResp = valib.Frequency(data=tdf, columns="target")
tResp.result.head(19)

xtbl,xcol,xval,xcnt,xpct
caso3_data_texto,target,E,690.0,0.3580063714757126
caso3_data_texto,target,H,26848.0,13.930079799101351
caso3_data_texto,target,N,2571.0,1.3339628711073293
caso3_data_texto,target,Q,8974.0,4.656158228439196
caso3_data_texto,target,M,753.0,0.3906939097408864
caso3_data_texto,target,F,1521.0,0.7891705666877665
caso3_data_texto,target,S,985.0,0.5110670665269231
caso3_data_texto,target,G,101849.0,52.84433467888385
caso3_data_texto,target,L,322.0,0.16706964002199923
caso3_data_texto,target,P,330.0,0.17122043853186256


In [12]:
con.execute('CREATE VIEW caso3 AS (SELECT ROW_NUMBER() OVER (ORDER BY detalle) - 1 AS doc_id, detalle, target FROM caso3_data_texto);')

In [13]:
tdf = DataFrame("caso3")

### Particion Muestral

Seleccionamos las variables más relevantes y dividimos en muestras 70% Train / 30% Test

In [14]:
## Seleccionamos variables más relevantes y generamos una columna que divide la tabla en muestras de entrenamiento y test
tbl_sample = tdf[['doc_id', 'detalle', 'target']].sample(frac = [0.5, 0.5])

In [15]:
## Almacenando la Muestra de Entrenamiento en la BD y generando la referencia con un DF de TeradataML
copy_to_sql(tbl_sample[tbl_sample.sampleid == "1"].drop("sampleid", axis = 1), schema_name="pocuser", table_name="TrainModel", if_exists="replace")
tbl_train = DataFrame("TrainModel")
tbl_train.shape

(96367, 3)

In [16]:
## Almacenando la Muestra de Test en la BD y generando la referencia con un DF de TeradataML
copy_to_sql(tbl_sample[tbl_sample.sampleid == "2"].drop("sampleid", axis = 1), schema_name="pocuser", table_name="TestModel", if_exists="replace")
tbl_test = DataFrame("TestModel")
tbl_test.shape

(96367, 3)

In [17]:
## Verificando que el DF hace referencia a la tabla creada en la BD
tbl_test.show_query()

'select * from "TestModel"'

In [18]:
stopwords = DataFrame("stopwords")

### Limpieza y Tokenización de Datos

In [19]:
TextParserTrain = TextParser(data=tbl_train,
                            text_column="detalle",
                            punctuation="\"<>!#$%&[]()*+,-./:;?@\^_`{|}~''",
                            object=stopwords,
                            remove_stopwords=True,
                            accumulate=["doc_id","target"])

In [20]:
TextParserTrain.result.head()

doc_id,target,token
0,G,mts
0,G,4
0,G,x
0,G,3
2,G,8
2,G,x
2,G,5
0,G,9
0,G,const
0,G,301036barra


Observamos que existen correlaciones significativas entre los grupos de variables históricas

### Entrenamiento del Modelo 

In [22]:
## Función para Entrenar el Modelo de Clasificación de Texto
NaiveBayesTextClassifierTrainer_out = NaiveBayesTextClassifierTrainer(data=TextParserTrain.result,
                                                                     token_column="token",
                                                                     doc_id_columns = 'doc_id',
                                                                     doc_category_column="target",
                                                                     model_type = "MULTINOMIAL",
                                                                     data_partition_column = "target")

In [23]:
## Métricas estadísticas del Modelo
NaiveBayesTextClassifierTrainer_out.result.head()

token,category,prob
0,N,0.00018334105295661571
0,E,0.0003248546275541695
0,H,0.00033780740473831185
0,Q,0.00445121328926606
0,J,4.763129566650472e-05
0,G,0.009836916607328593
0,A,0.0006629192055866835
0,F,1.863811307743204e-05
0,I,4.221119103096613e-05
0,O,0.0005592111394858985


In [24]:
NaiveBayesTextClassifierTrainer_out.model_data.head()

token,category,prob
0,J,4.763129566650472e-05
0,N,0.00018334105295661571
0,Q,0.00445121328926606
0,O,0.0005592111394858985
0,A,0.0006629192055866835
0,I,4.221119103096613e-05
0,G,0.009836916607328593
0,C,0.008884043704288259
0,H,0.00033780740473831185
0,E,0.0003248546275541695


In [25]:
TextParserTest = TextParser(data=tbl_test,
                            text_column="detalle",
                            punctuation="\"<>!#$%&[]()*+,-./:;?@\^_`{|}~''",
                            object=stopwords,
                            remove_stopwords=True,
                            accumulate=["doc_id","target"])

In [26]:
TextParserTest.result.head()

doc_id,target,token
2,G,8
2,G,mts
2,G,301036barra
2,G,const
7,G,kn95
7,G,mayfield
7,G,cdata
2,G,x
2,G,9
2,G,5


### Scoring del Modelo

In [51]:
nbt_predict_out = NaiveBayesTextClassifierPredict(object = NaiveBayesTextClassifierTrainer_out.result,
                                                      newdata = TextParserTest.result,
                                                      input_token_column = 'token',
                                                      doc_id_columns = 'doc_id',
                                                      model_type = "MULTINOMIAL",
                                                      model_token_column = 'token',
                                                      model_category_column = 'category',
                                                      accumulate = 'target',
                                                      responses = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S'],
                                                      output_prob = True,
                                                      model_prob_column = 'prob',
                                                      newdata_partition_column = 'doc_id')

In [53]:
nbt_predict_out.result.head()

doc_id,prediction,loglik_A,loglik_B,loglik_C,loglik_D,loglik_E,loglik_F,loglik_G,loglik_H,loglik_I,loglik_J,loglik_K,loglik_L,loglik_M,loglik_N,loglik_O,loglik_P,loglik_Q,loglik_R,loglik_S,prob_A,prob_B,prob_C,prob_D,prob_E,prob_F,prob_G,prob_H,prob_I,prob_J,prob_K,prob_L,prob_M,prob_N,prob_O,prob_P,prob_Q,prob_R,prob_S,target
8,G,-86.20437860899557,-87.2670107149929,-87.57999625026703,-87.60765733060346,-83.74798682379388,-86.17298306978813,-80.599177061873,-89.52948336219275,-86.37002775330112,-86.16293543745284,-88.87890849913725,-86.90912778958126,-87.14513022118244,-86.16605427033318,-86.17288170305046,-86.7512133604433,-86.60671913491979,-86.22336396161107,-86.70916258130586,0.0033995355012297885,0.0011746928380623892,0.0008590064603315253,0.0008355710325025623,0.03964760688246285,0.0035079588503534012,0.9241185387707684,0.00012227704498862346,0.0028805742329045692,0.0035433831985207795,0.0002343613590968672,0.0016801610037993381,0.001326955463265864,0.003532349194045682,0.0035083144587210566,0.001967579394647078,0.0022734494942961217,0.003335602932519289,0.0020520818874837873,G
11,G,-813.3709155683786,-889.0120709115386,-766.1637791750784,-867.942285035776,-873.8933339948995,-874.0808857447917,-725.0667188944251,-847.7876896292075,-888.6135509086205,-883.0648937953495,-899.0027408255249,-897.7429649454391,-889.5573154221089,-815.2539346221323,-821.4463041591288,-899.5192516918333,-884.6419828631858,-911.2837185145876,-872.7752867547405,4.4665752551495414e-39,6.301417943424727e-72,1.4183176181201245e-18,8.911073503964687e-63,2.319649146160424e-65,1.9229571707282195e-65,1.0,5.046143493513181e-54,9.386708331864939e-72,2.411373532531345e-69,2.887656104201184e-76,1.0177923546884705e-75,3.6529321281620847e-72,6.79500641755103e-40,1.3895620715266492e-42,1.722771394765327e-76,4.981308167721244e-70,1.3396292711388142e-81,7.095517057638907e-65,G
14,C,-231.9955719319169,-292.9704081783676,-196.21736634599907,-292.49343490079565,-269.9055753675675,-284.4065207783412,-200.55173616171479,-254.11591207987527,-280.8126102984064,-289.70483940386623,-294.2624152861088,-292.8240907093784,-292.21544022811895,-266.76427463915326,-271.57380759444766,-270.1532403126463,-274.34648849072227,-289.848417248872,-283.64581904278464,2.8580253883293193e-16,9.441286258826881e-43,0.9870595181710963,1.5211709675863315e-42,9.816592909783959e-33,4.9462971945716856e-39,0.012940481828903576,7.068453711200376e-26,1.7992673407120318e-37,2.4731454836505886e-41,2.5936992002669714e-43,1.0928889501829699e-42,2.0086745654056327e-42,2.2709645437656535e-31,1.851214446134859e-33,7.663042996444563e-33,1.1569023875586224e-34,2.1423705950329108e-41,1.058397421636148e-38,G
16,C,-382.69478015485515,-474.6514068302638,-322.8003891494965,-475.4404884046533,-434.98308347020895,-453.4546231094449,-328.99296953170284,-396.93437795613715,-435.5292903216534,-464.79468555073504,-482.95761259235144,-473.3162368889169,-470.4821224509865,-408.22224518672624,-416.2040547248789,-428.33484248461946,-420.46289246975175,-473.64555038284544,-472.9378724522124,9.71201761917986e-27,1.124744613699261e-66,0.9979596273968585,5.109285233706835e-67,1.9001499022791466e-49,1.8059267297515358e-57,0.0020403726031415067,6.355221796689713e-33,1.10045741760129e-49,2.146711008553275e-62,2.7778871193662818e-70,4.274751368095293e-66,7.273628595834811e-65,7.959252111057628e-38,2.7190451758098625e-41,1.465825383616775e-46,3.84438338270381e-43,3.075330721735036e-66,6.240687657874301e-66,G
21,H,-62.98574485257002,-64.41107766350683,-58.752293834606476,-64.76555458051988,-64.21353101957222,-63.00608474150343,-60.439186216035054,-57.422542301587015,-63.00550156488935,-63.04001853102159,-66.05851885614042,-64.02968746515937,-64.28252386701618,-60.76626629449373,-63.00622568491373,-63.856339041249214,-63.09684338607873,-62.9822272136399,-63.80938295339105,0.0027732050102182333,0.0006667573341061164,0.19122575868594968,0.0004677571475639521,0.0008123837334891995,0.002717368111495406,0.03539470848972593,0.722853262

### Evaluando las Clasificaciones

In [57]:
predicted_data = ConvertTo(data = nbt_predict_out.result,
                               target_columns = ["target", "prediction"],
                               target_datatype = ["VARCHAR(charlen=20,charset=UNICODE,casespecific=NO)"])

In [59]:
ClassificationEvaluator_obj = ClassificationEvaluator(data=predicted_data.result,
                                                          observation_column='target',
                                                          prediction_column='prediction',
                                                          labels=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S'])

In [ ]:
ClassificationEvaluator_obj.result.head(22).sort('SeqNum')

In [ ]:
ClassificationEvaluator_obj.output_data.sort('SeqNum')

In [62]:
copy_to_sql(predicted_data.result, schema_name="pocuser", table_name="matriz_clas", if_exists="replace")

## **Finalizando la Demo**

In [ ]:
con.execute("DROP VIEW pocuser.CASO3;")

In [ ]:
con.execute("DROP TABLE pocuser.TrainModel;")

In [ ]:
con.execute("DROP TABLE pocuser.TestModel;")

In [ ]:
con.execute("DROP TABLE pocuser.pmml_models;")

In [44]:
con.execute("DROP TABLE pocuser.matriz_clas;")

In [ ]:
## Finalizando el Notebook y Limpiando el ambiente 
remove_context()

![Slide](images/Diapositiva13.PNG)

![Slide](images/Diapositiva14.PNG)

Copyright 2023. Elaborado por Luis Cajachahua bajo licencia MIT